In [1]:
import numpy as np
from numba import jit

In [2]:
file_input = 1308
test_input = 8

inp = test_input

In [6]:
@jit(nopython=True)
def getPowerLevel(x, y, serial_number=inp):
    rackID = x + 10
    pl = rackID * y + serial_number
    pl *= rackID
    # keep the hundreds digit
    pl = (pl // 100) % 10
    return pl - 5

In [7]:
assert(getPowerLevel(3, 5, 8) == 4)
assert(getPowerLevel(122, 79, 57) == -5)
assert(getPowerLevel(217, 196, 39) == 0)
assert(getPowerLevel(101, 153, 71) == 4)

In [25]:
def genGrid(serial_number, ngrid=300, window_size=3):
    xx, yy = np.meshgrid(np.arange(1, ngrid+1), np.arange(1, ngrid+1))
    grid = getPowerLevel(xx, yy, serial_number)
    sum_grid = np.zeros((ngrid, ngrid))
    for i in range(ngrid-window_size+1):
        for j in range(ngrid-window_size+1):
            sum_grid[i, j] = grid[i:i+window_size, j:j+window_size].sum()
    return grid, sum_grid

### Test
For grid serial number 18, the largest total 3x3 square has a top-left corner of 33,45 (with a total power of 29)

In [16]:
%time
grid, sum_grid = genGrid(18)     

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 12.4 µs


In [17]:
y, x = np.unravel_index(np.argmax(sum_grid), sum_grid.shape)
print(x+1, y+1, sum_grid[y, x])

33 45 29.0


For grid serial number 42, the largest 3x3 square's top-left is 21,61 (with a total power of 30)

In [18]:
grid, sum_grid = genGrid(42)
y, x = np.unravel_index(np.argmax(sum_grid), sum_grid.shape)
print(x+1, y+1, sum_grid[y, x])

21 61 30.0


# Part A

In [19]:
grid, sum_grid = genGrid(file_input)
y, x = np.unravel_index(np.argmax(sum_grid), sum_grid.shape)
print(x+1, y+1, sum_grid[y, x])
#print ("X: {} Y: {} | total power {}".format(xi, yi, sum_grid[yi-1, xi-1]))

21 41 31.0


# Part B

In [26]:
def genGridMult(serial_number, ngrid=300):
    xx, yy = np.meshgrid(np.arange(1, ngrid+1), np.arange(1, ngrid+1))
    grid = getPowerLevel(xx, yy, serial_number)
    
    sum_grid = np.zeros((ngrid, ngrid, ngrid))
    for i in range(ngrid):
        for j in range(ngrid):            
            subgrid = grid[i:, j:]
            cumsum = subgrid.cumsum(axis=1).cumsum(axis=0)
            window_sums = cumsum.diagonal()
            sum_grid[:len(window_sums), i, j] = window_sums
    return grid, sum_grid

In [27]:
def findBestWindow(serial_number,ngrid=300):
    grid, sum_grid = genGridMult(serial_number, ngrid=ngrid)
    w, y, x = np.unravel_index(np.argmax(sum_grid, axis=None), sum_grid.shape)
    return (x+1, y+1, w+1)

For grid serial number 18, the largest total square (with a total power of 113) is 16x16 and has a top-left corner of 90,269, so its identifier is 90,269,16

In [28]:
%%time
findBestWindow(18)

CPU times: user 13.2 s, sys: 84 ms, total: 13.3 s
Wall time: 13.6 s


(90, 269, 16)

For grid serial number 42, the largest total square (with a total power of 119) is 12x12 and has a top-left corner of 232,251, so its identifier is 232,251,12

In [263]:
%%time
findBestWindow(42)

CPU times: user 13.2 s, sys: 91 ms, total: 13.3 s
Wall time: 13.5 s


(232, 251, 12)

Solution:

In [264]:
%%time
findBestWindow(file_input)

CPU times: user 13.7 s, sys: 97 ms, total: 13.8 s
Wall time: 14.1 s


(227, 199, 19)

In [16]:
grid, sum_grid = genGridMult(file_input)

In [22]:
grid

array([[ 0,  3, -4, ...,  2,  1,  0],
       [ 1,  4, -2, ...,  0, -4,  1],
       [ 2, -4,  0, ..., -1,  1,  2],
       ...,
       [-1,  1, -2, ..., -2,  0, -3],
       [ 0,  2,  0, ..., -3,  4, -2],
       [ 1,  3,  2, ..., -5, -1, -1]])

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

In [17]:
for i in range(195, 300):
    fig = plt.figure(figsize=(12,8))
    plt.title("{:03}".format(i))
    plt.imshow(sum_grid[i])
    plt.colorbar()
    plt.savefig('images/image_{:03d}.png'.format(i+1))
    plt.close(fig)